# Razones de No Solución Qx

## Dependencias

In [1]:
import pandas as pd
import numpy as np
import re
import unicodedata
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk

import spacy
from spacy.lang.es import Spanish
from spacy.lang.es.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer

from varclushi import VarClusHi

## Funciones

In [2]:
def clean_text(text, pattern="[^a-zA-Z ]"):
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, "", cleaned_text.decode("utf-8"), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().split())
    return cleaned_text

In [3]:
def tokestem(x):
    stemmer = SnowballStemmer('spanish')
    a=word_tokenize(x)
    a=[stemmer.stem(i) for i in a]
    return a

## Lectura de Archivo

In [291]:
df=pd.read_pickle('No_solucion.pkl')

In [292]:
df

,linea,fecha,no_sol
79,7225557236,2021-03-23,No tenían el equipo en existencia
100,5521287428,2021-10-14,"Quiero renovar mi plan, pero Requiero el equip..."
540,5540571174,2021-06-06,Por qué al parecer el sistema no le permitía a...
593,5520174180,2021-09-29,"Atencion del asesor, nunca estuvo dispuesto a ..."
696,7773040403,2021-10-03,Falta de claridad y de equipos\n
...,...,...,...
569638,5528257566,2021-09-13,Solo me comentaron que ya no podría adquirir u...
569982,5537079349,2021-01-07,Fui hacer la cancelación de mi plan por que ot...
569987,5531065286,2021-09-20,No hubo señal de internet y no recibí atención...
570031,5531151067,2021-02-13,"El Telcel up es un engaño, roban el dinero y n..."


## Stemming, Lematización

In [5]:
stemmer = SnowballStemmer('spanish')

In [6]:
nlp = spacy.load('es_core_news_sm')

In [96]:
# Lemma + Stemming
#df['text']=df['no_sol'].str.replace("\n", "").map(nlp).map(lambda x:[w.lemma_ for w in x]).apply(" ".join).map(clean_text).map(tokestem).str.join(" ")
# Stemming
df['text']=df['no_sol'].str.replace("\n", "").map(clean_text).map(tokestem).str.join(" ")

## Stop Words

In [155]:
sw=list(STOP_WORDS.union(set(stopwords.words('spanish'))))

In [151]:
sw=list(set([w.lemma_ for w in nlp(" ".join(sw))]))

In [156]:
sw=list(pd.Series(sw).map(clean_text))

In [157]:
stemmer = SnowballStemmer('spanish')
sw=list(set([stemmer.stem(i) for i in sw]))

## Modificaciones

In [201]:
sw.append('pod')
#sw.append('quer')
sw.append('renov')
sw.append('renovacion')
sw.append('dec')
sw.append('dar')
sw.append('telcel')
#sw.append('equip')
#sw.append('ten')
#sw.append('cotiz')
sw.append('apart')
sw.append('no')
sw.append('quer')
sw.append('centr')
sw.append('plan')
sw.append('cambi')
sw.append('queri')

In [159]:
sw.remove('hab')

## Stop Words Check 

In [342]:
Sw=pd.Series(sw)

In [343]:
Sw[Sw.str.contains('cuo')]

Series([], dtype: object)

In [344]:
sw

['ayer',
 'hubim',
 'ejempl',
 'trabaj',
 'con',
 'qued',
 'pod',
 'haz',
 'bastant',
 'hayais',
 'hub',
 'rar',
 'pes',
 'sea',
 'dias',
 'much',
 'asegur',
 'usais',
 'sois',
 'dich',
 'respect',
 'cinc',
 'cuatr',
 'tempran',
 'habeis',
 'habl',
 'grand',
 'nuev',
 'ultim',
 'parec',
 'ahor',
 'el',
 'trabajai',
 'hor',
 'tuvies',
 'cuand',
 'estariais',
 'habr',
 'algo',
 'haci',
 'pues',
 'hoy',
 'todavi',
 'maner',
 'nosotr',
 'embarg',
 'tuvieram',
 'o',
 'tras',
 'conoc',
 'lueg',
 'trat',
 'podei',
 'estais',
 'emple',
 'arrib',
 'fuim',
 'sereis',
 'demasi',
 'bien',
 'adem',
 'ocho',
 'adel',
 'baj',
 'general',
 'podriais',
 'que',
 'delant',
 'fuisteis',
 'vari',
 'habreis',
 'segun',
 'menud',
 'este',
 'ir',
 'contr',
 'manifest',
 'hubies',
 'asi',
 'mientr',
 'peor',
 'part',
 'hubiesei',
 'a',
 'soy',
 'tampoc',
 'hast',
 'expres',
 'cualqu',
 'agreg',
 'tant',
 'encim',
 'ya',
 'principal',
 'dem',
 'clar',
 'anad',
 'igual',
 'entonc',
 'estari',
 'pon',
 'trav',
 '

In [345]:
sw.remove('tiemp')

ValueError: list.remove(x): x not in list

In [346]:
for i in 'des hac renovacion anticip de mi mism plan con cambi de equip y ahor'.split():
    sw.append(i)

In [371]:
sw.remove('equip')

In [372]:
sw.remove('ten')

In [350]:
for i in 'venc el proxim mes'.split():
    sw.append(i)

In [351]:
sw.remove('estar')

ValueError: list.remove(x): x not in list

In [352]:
sw.append('uno')

## TFIDF

In [202]:
#vect = TfidfVectorizer(min_df=70, ngram_range=(1,3))
vect = TfidfVectorizer(min_df=100, ngram_range=(1,3), stop_words=list( sw))

In [203]:
X=vect.fit_transform(df['text'])

In [204]:
X.shape

(5360, 59)

vc = VarClusHi(df=pd.DataFrame(X.todense(),columns=vect.get_feature_names()),feat_list=vect.get_feature_names())
vc.varclus()
rs = vc.rsquare.sort_values(by=['Cluster','RS_Ratio'])
best_cv = rs.groupby('Cluster').first()['Variable'].to_list()


len(best_cv)

X=pd.DataFrame(X.todense(),columns=vect.get_feature_names())

X[best_cv]

## Singular Value Descomposition

In [205]:
from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=7, algorithm='randomized', n_iter=100, random_state=0)

svd_model.fit(X)

len(svd_model.components_)

7

## Tópicos

In [206]:
terms = vect.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])
        print(" ")

Topic 0: 
equip
 
asesor
 
pag
 
sistem
 
ofrec
 
dispon
 
compr
 
Topic 1: 
sistem
 
asesor
 
informacion
 
atencion
 
servici
 
pag
 
tramit
 
Topic 2: 
sistem
 
equip
 
problem
 
financi
 
esper
 
enganch
 
adquir equip
 
Topic 3: 
asesor
 
sistem
 
equip
 
existent
 
inter
 
groser
 
sup
 
Topic 4: 
informacion
 
falt
 
solicit
 
existent
 
equip
 
atencion
 
dispon
 
Topic 5: 
atencion
 
pesim
 
client
 
servici
 
equip
 
actitud
 
groser
 
Topic 6: 
ofrec
 
opcion
 
telefon
 
informacion
 
promocion
 
sistem
 
cost
 


## Tabla Texto-Tópico

In [207]:
aux=pd.concat([pd.Series(np.argmax(svd_model.transform(X),axis=1)),pd.Series(np.sum(svd_model.transform(X),axis=1)),df['text'].reset_index(drop=True),df['no_sol'].reset_index(drop=True)],axis=1)

In [208]:
aux.columns=['Tema','aux','Texto','Texto1']

In [209]:
aux['Tema1']=pd.Series(map(lambda x,y: 7 if y==0 else x, aux['Tema'],aux['aux']))

## Documentos por tópico

In [283]:
aux['Tema1'].value_counts()

0    1872
1    1599
7     560
6     412
5     292
4     283
2     222
3     120
Name: Tema1, dtype: int64

## Validación

In [301]:
aux[aux['Tema1']==5]['Texto1'].sample(15).values

array(['No los conté exactamente pero más o menos serían 20 empleados y tres o cuatro clientes \nDefinitivamente la atención es nefasta \nY creo les falta preparación y amabilidad!!',
       'Por que llevo varios días buscando un equipo de la marca samsum y en ni gun sentro de atención hay y ni fecha tienen de entrega ',
       'Al momento de cambiar el titular elevaron el precio del plan sin ningún beneficio extra y la atención de la persona que atendía no se veía interesada en ayudarme a resolver el problema ',
       'Pésimo servicio en general ! Cobros excesivos y la red es pésima ',
       'Tuve que salir de la tienda por la pésima atención de la Gerente de la tienda grosera y prepotente le falto al respeto a mi familia son un asco de sucursal por culpa de personal ',
       'Se supone que ya puedo renovar plan y el miércoles me dijeron en un centro de atención que no se podía conteste está encuesta y me llaman  el jueves de esta semana para decirme que sí puedo renovar y al acudi

In [285]:
aux

,Tema,aux,Texto,Texto1,Tema1
0,0,0.505889,no teni el equip en existent,No tenían el equipo en existencia,0
1,0,0.441241,quier renov mi plan per requier el equip xiaom...,"Quiero renovar mi plan, pero Requiero el equip...",0
2,2,1.683986,por que al parec el sistem no le permiti actua...,Por qué al parecer el sistema no le permitía a...,2
3,1,0.544591,atencion del asesor nunc estuv dispuest a acla...,"Atencion del asesor, nunca estuvo dispuesto a ...",1
4,0,0.271735,falt de clarid y de equip,Falta de claridad y de equipos\n,0
...,...,...,...,...,...
5355,0,0.258562,sol me coment que ya no podri adquir un equip ...,Solo me comentaron que ya no podría adquirir u...,0
5356,1,0.009696,fui hac la cancelacion de mi plan por que otra...,Fui hacer la cancelación de mi plan por que ot...,1
5357,5,0.407636,no hub senal de internet y no recibi atencion ...,No hubo señal de internet y no recibí atención...,5
5358,3,0.361208,el telcel up es un engan rob el diner y no cum...,"El Telcel up es un engaño, roban el dinero y n...",3


In [294]:
df=df.reset_index(drop=True)

In [295]:
df['Tema']=aux['Tema1']

In [303]:
df['Tema']=df['Tema'].replace({0:'Falta de Equipo/Equipo Forzoso',1:'Asesor Mal Informado/Burocracia',2:'Fallas en el Sistema',3:'Actitud',4:'Malos Entendidos',5:'Mala Atención/Mal Servicio',6:'No Hubo Oferta de Alternativas',7:'No Clasificado'})

In [304]:
df.index=pd.read_pickle('No_solucion.pkl').index

In [306]:
df.to_pickle('Temas_No_sol.pkl')

In [214]:
aux['Texto1'].loc[3252]

'Comentaron que por las nuevas políticas de la empresa aún no puedo renovar mi plan aunque quiera dar el monto de penalización para renovarlo cuando yo quiera'

In [147]:
aux['Texto'].loc[2225]

'no hay promocion cad ves mas car'

In [273]:
aux[aux['Texto1']=='Me querían cobrar una cuota por renovar mi plan anticipadamente ya qué vence el próximo mes ']

,Tema,aux,Texto,Texto1,Tema1
2680,0,-0.143969,yo quer cobr uno cuot por renov mi plan antici...,Me querían cobrar una cuota por renovar mi pla...,0


In [ ]:
X=X[best_cv]

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

In [ ]:
l = []
for k in range(2,10):
    cl = GaussianMixture(n_components=k)
    cl.fit(X)
    l.append((k,silhouette_score(X,cl.predict(X))))
sil = pd.DataFrame(l,columns=['k','sil']).set_index('k')
sil.iplot(mode='lines+markers')

In [ ]:
k = 5

In [ ]:
cl = GaussianMixture(n_components=k)
cl.fit(X)

In [ ]:
aux['cl_cv'] = cl.predict(X)

In [ ]:
aux["cl_cv"].value_counts(True).reset_index().iplot(kind="pie", labels='index',values='cl_cv',hole=.7, title='Clientes por Cluster')

In [ ]:
aux[aux['cl_cv']==1]['Tema1'].value_counts()

In [ ]:
aux[aux['cl_cv']==0]['Tema1'].value_counts()

In [ ]:
aux[aux['cl_cv']==3]['Tema1'].value_counts()

In [ ]:
aux[aux['cl_cv']==2]['Tema1'].value_counts()

In [ ]:
aux[aux['cl_cv']==4]['Tema1'].value_counts()